In [4]:
from transformers import pipeline, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline
from time import time

## Emotions - EN: https://huggingface.co/SamLowe/roberta-base-go_emotions
## Emotions - CS: https://huggingface.co/fremy7/xlm_roberta_emotion_detector
## 28 ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 
## 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 
## 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']

## Intents: https://huggingface.co/bespin-global/klue-roberta-small-3i4k-intent-classification?text=What+a+nice+day%21
## 0: "fragment",
## 1: "statement",
## 2: "question",
## 3: "command",
## 4: "rhetorical question",
## 5: "rhetorical command",
## 6: "intonation-dependent utterance"

## Sentiment: https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest
## positive
## neutral
## negative

MODEL_EMOTIONS = {
    'en': 'SamLowe/roberta-base-go_emotions',
    'cs': 'fremy7/xlm_roberta_emotion_detector'
}

MODEL_INTENTS = {
    'en': 'bespin-global/klue-roberta-small-3i4k-intent-classification'
}

MODEL_SENTIMENT = {
    'en': 'cardiffnlp/twitter-roberta-base-sentiment-latest'
}


class Text2Vec4TTS:
    
    def __init__(self, language='en'):
        self.language = language

        
        self.clf_emotion = pipeline(
            task='text-classification', 
            model=MODEL_EMOTIONS[self.language], 
            top_k=None
        ) if self.language in MODEL_EMOTIONS else None
        
        self.clf_intent = TextClassificationPipeline(
            tokenizer=RobertaTokenizerFast.from_pretrained(MODEL_INTENTS[self.language]), 
            model=RobertaForSequenceClassification.from_pretrained(MODEL_INTENTS[self.language]), 
            return_all_scores=True
        ) if self.language in MODEL_INTENTS else None
            
        self.clf_sentiment = pipeline(
            task='sentiment-analysis', 
            model=MODEL_SENTIMENT[self.language], 
            tokenizer=MODEL_SENTIMENT[self.language], 
            return_all_scores=True
        ) if self.language in MODEL_SENTIMENT else None
        
    def to_vec(
        self,
        text,
        intent=True,
        emotion=True,
        sentiment=True
    ):
        vec = []
        info = {
            'intent': {},
            'emotion': {},
            'sentiment': {}
        }
        t0 = time()
        
        if intent and self.clf_intent:
            preds_intent = self.clf_intent(text)[0]
            vec.extend([inta['score'] for inta in sorted(preds_intent, key=lambda x:x['label'])])
            
            for inta in sorted(preds_intent, key=lambda x:-x['score'])[:3]:
                info['intent'].update({inta['label']:inta['score']})
                
        if emotion and self.clf_emotion:
            preds_emotion = self.clf_emotion(text)[0]
            vec.extend([em['score'] for em in sorted(preds_emotion, key=lambda x:x['label'])])
            
            for em in sorted(preds_emotion, key=lambda x:-x['score'])[:3]:
                info['emotion'].update({em['label']:em['score']})
                
        if sentiment and self.clf_sentiment:
            preds_sentiment = self.clf_sentiment(text)[0]
            vec.extend([sent['score'] for sent in sorted(preds_sentiment, key=lambda x:x['label'])])
            
            for sent in sorted(preds_sentiment, key=lambda x:-x['score'])[:3]:
                info['sentiment'].update({sent['label']:sent['score']})
            
        info.update({'t': time()-t0})
        return vec, info

In [ ]:
## Create the model (language selection)

model = Text2Vec4TTS(language='cs')  ## CZECH / MULTILINGUAL

#model = Text2Vec4TTS(language='en')  ## ENGLISH

In [ ]:
## CZECH / MULTILINGUAL

text = 'Jsem velmi nešťastný.'

vec, info = model.to_vec(
        text, 
        intent=True, 
        emotion=True, 
        sentiment=True
    )

print(vec)
print(info)

In [ ]:
## ENGLISH

for text in (
    'What the hell! I hate those disgusting people.',
    'What a beautiful day!'
):
    print(f'\n:: {text}')
    vec, info = model.to_vec(
        text, 
        intent=True, 
        emotion=True, 
        sentiment=True
    )

    print(f'Vector length: {len(vec)}')
    for k, v in info.items():
        if k == 't':
            print(f'\t{k}: {round(v, 2)}s')
        else:
            print(f'\t{k}')
            for kk, sc in v.items():
                print(f'\t\t{kk}: {round(sc, 4)}')